# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd
#Task 1.2
from pyspark.sql.functions import expr, percentile_approx
from pyspark.sql.functions import abs
#Task 3.1
from pyspark.sql.functions import sum,avg,max,count,mean,col,desc
from pyspark.sql.functions import lit

In [0]:
#TASK 0
# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

#TASK 1
## solution implementation to task 1.1 goes HERE
def t11_remove_zeros(data):
    # input: trips dataset
    data = data.where((trips.trip_distance!=0) | ((trips.passenger_count != 0) & (trips.total_amount != 0)))
    return data

## solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    #Total Amount Percentile calculation
    MedTotalAmount = df.agg(percentile_approx("total_amount",0.5).alias('median'))
    df = df.withColumn("TotalAmountMedDifference",abs(df["total_amount"]-MedTotalAmount.collect()[0][0]))
    df1 = df.agg(percentile_approx("TotalAmountMedDifference",0.5).alias('median'))
    df = df.withColumn("Zscore",df["TotalAmountMedDifference"]/1.483*df1.collect()[0][0])
    df = df.where(df.Zscore < '3.5')
    #Trip distance Percentile calculation
    MedTripDist = df.agg(percentile_approx("trip_distance",0.5).alias('median'))
    df = df.withColumn("TripDistanceMedDifference",abs(df["trip_distance"]-MedTripDist.collect()[0][0]))
    df1 = df.agg(percentile_approx("TripDistanceMedDifference",0.5).alias('median'))
    df = df.withColumn("Zscore",df["TripDistanceMedDifference"]/1.483*df1.collect()[0][0])
    df = df.where(df.Zscore < '3.5')
    return df

#TASK 2
## solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    df = df.join(zones_df,df["PULocationID"] == zones_df["LocationID"],"left" )\
    .drop("Borough")\
    .drop("service_zone")\
    .drop("LocationID")\
    .withColumnRenamed("Zone","PUZone")\
    .join(zones_df,df["DOLocationID"] == zones_df["LocationID"],"left" )\
    .drop("Borough")\
    .drop("service_zone")\
    .drop("LocationID")\
    .withColumnRenamed("Zone","DOZone")\
    
    return df

## solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn("unit_profitability",pf.round(df.total_amount / df.trip_distance))
    return df

#TASK 3
## solution implementation to task 3.1 goes HERE

def t31_summarise_trips(df):
    # input: output of task 2.2
    df2 = df.withColumn("trips_count", lit(1)) \
    .groupBy("PUZone", "DOzone") \
    .agg({"unit_profitability": "avg",
          "trips_count": "sum",
          "passenger_count": "sum"}) \
    .withColumnRenamed("avg(unit_profitability)", "average_unit_profit") \
    .withColumnRenamed("sum(trips_count)", "trips_count") \
    .withColumnRenamed("sum(passenger_count)", "total_passengers")

    
    df2.select(sum(df2.trips_count)).show()
    return df2

## solution implementation to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    # input: output of task 2.2
    df = df.groupby("PUZone")\
    .agg(sum("trips_count").alias("trips_count"),\
    mean("average_unit_profit").alias("average_unit_profit"),\
    sum("total_passengers").alias("total_passengers"))
    return df
    
# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.sort(desc("trips_count"))
    df_zones = df_zones.drop("average_unit_profit","total_passengers")
    return df_zones.take(10)

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    
    df_zones = df_zones.sort(desc("average_unit_profit"))
    df_zones = df_zones.drop("total_passengers","trips_count")
    return df_zones.take(10)

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    #return Non
   
    df_zones = df_zones.sort(desc("total_passengers"))
    df_zones = df_zones.drop("average_unit_profit","trips_count")
    return df_zones.take(10)

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))


In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# execute task 1.1
trips=trips.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 2,890,208


In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 419,933


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
display(trips_21.take(10))

Row count: 419,933


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,TotalAmountMedDifference,Zscore,TripDistanceMedDifference,PUZone,DOZone
114,48,2.0,1.0,14.75,0.5500000000000007,0.023600809170600145,0.10000000000000009,Greenwich Village South,Clinton East
234,148,1.8,1.0,14.75,0.5500000000000007,0.02360080917060009,0.09999999999999987,Union Sq,Lower East Side
249,79,1.15,1.0,14.16,1.1400000000000006,0.17700606877950095,0.75,West Village,East Village
48,90,2.16,1.0,14.3,1.0,0.061362103843560376,0.26000000000000023,Clinton East,Flatiron
13,246,2.34,1.0,15.96,0.6600000000000001,0.10384356035064053,0.43999999999999995,Battery Park City,West Chelsea/Hudson Yards
48,246,1.9,2.0,14.75,0.5500000000000007,0.0,0.0,Clinton East,West Chelsea/Hudson Yards
162,114,2.02,1.0,14.16,1.1400000000000006,0.028320971004720173,0.1200000000000001,Midtown East,Greenwich Village South
229,262,1.98,4.0,14.76,0.5400000000000009,0.01888064733648012,0.08000000000000007,Sutton Place/Turtle Bay North,Yorkville East
138,92,3.8,1.0,16.05,0.75,0.4484153742414023,1.9,LaGuardia Airport,Flushing
239,42,3.22,2.0,15.8,0.5,0.3115306810519217,1.3200000000000003,Upper West Side South,Central Harlem North


In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
display(trips_22.take(10))

Row count: 419,933


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,TotalAmountMedDifference,Zscore,TripDistanceMedDifference,PUZone,DOZone,unit_profitability
114,48,2.0,1.0,14.75,0.5500000000000007,0.023600809170600145,0.10000000000000009,Greenwich Village South,Clinton East,7.0
234,148,1.8,1.0,14.75,0.5500000000000007,0.02360080917060009,0.09999999999999987,Union Sq,Lower East Side,8.0
249,79,1.15,1.0,14.16,1.1400000000000006,0.17700606877950095,0.75,West Village,East Village,12.0
48,90,2.16,1.0,14.3,1.0,0.061362103843560376,0.26000000000000023,Clinton East,Flatiron,7.0
13,246,2.34,1.0,15.96,0.6600000000000001,0.10384356035064053,0.43999999999999995,Battery Park City,West Chelsea/Hudson Yards,7.0
48,246,1.9,2.0,14.75,0.5500000000000007,0.0,0.0,Clinton East,West Chelsea/Hudson Yards,8.0
162,114,2.02,1.0,14.16,1.1400000000000006,0.028320971004720173,0.1200000000000001,Midtown East,Greenwich Village South,7.0
229,262,1.98,4.0,14.76,0.5400000000000009,0.01888064733648012,0.08000000000000007,Sutton Place/Turtle Bay North,Yorkville East,7.0
138,92,3.8,1.0,16.05,0.75,0.4484153742414023,1.9,LaGuardia Airport,Flushing,4.0
239,42,3.22,2.0,15.8,0.5,0.3115306810519217,1.3200000000000003,Upper West Side South,Central Harlem North,5.0


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
#display(graph.take(10))

+----------------+
|sum(trips_count)|
+----------------+
|          419933|
+----------------+

Row count: 5,074


In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_trips

Out[63]: [Row(PUZone='Upper East Side South', trips_count=19305),
 Row(PUZone='Midtown Center', trips_count=18716),
 Row(PUZone='Upper East Side North', trips_count=18035),
 Row(PUZone='Penn Station/Madison Sq West', trips_count=17669),
 Row(PUZone='Murray Hill', trips_count=16670),
 Row(PUZone='Midtown East', trips_count=16290),
 Row(PUZone='Lincoln Square East', trips_count=15662),
 Row(PUZone='Midtown North', trips_count=13789),
 Row(PUZone='Upper West Side South', trips_count=13472),
 Row(PUZone='Union Sq', trips_count=13271)]

In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
top10_profit

Out[64]: [Row(PUZone='Prospect Park', average_unit_profit=389.0),
 Row(PUZone='Queensboro Hill', average_unit_profit=303.9166666666667),
 Row(PUZone='Van Cortlandt Village', average_unit_profit=151.61698717948718),
 Row(PUZone='Pelham Bay', average_unit_profit=87.66666666666667),
 Row(PUZone='South Ozone Park', average_unit_profit=66.12142857142858),
 Row(PUZone='Woodside', average_unit_profit=40.23650793650794),
 Row(PUZone='Mott Haven/Port Morris', average_unit_profit=36.536231884057976),
 Row(PUZone='Parkchester', average_unit_profit=33.59375),
 Row(PUZone='Garment District', average_unit_profit=33.19223738182151),
 Row(PUZone='Hammels/Arverne', average_unit_profit=30.0)]

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_passenger

Out[65]: [Row(PUZone='Upper East Side South', total_passengers=27073.0),
 Row(PUZone='Midtown Center', total_passengers=26924.0),
 Row(PUZone='Upper East Side North', total_passengers=25299.0),
 Row(PUZone='Penn Station/Madison Sq West', total_passengers=25289.0),
 Row(PUZone='Murray Hill', total_passengers=23677.0),
 Row(PUZone='Midtown East', total_passengers=23232.0),
 Row(PUZone='Lincoln Square East', total_passengers=22477.0),
 Row(PUZone='Midtown North', total_passengers=19790.0),
 Row(PUZone='Union Sq', total_passengers=19515.0),
 Row(PUZone='East Village', total_passengers=19494.0)]

## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'delta'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: S
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-S-delta/
      Number of trips (dataset rows): 2,898,033
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

+----------------+
|sum(trips_count)|
+----------------+
|         2890208|
+----------------+

Out[67]: [[Row(PUZone='Upper East Side South', trips_count=127440),
  Row(PUZone='Upper East Side North', trips_count=109554),
  Row(PUZone='JFK Airport', trips_count=108677),
  Row(PUZone='Penn Station/Madison Sq West', trips_count=103471),
  Row(PUZone='Midtown Center', trips_count=99210),
  Row(PUZone='Murray Hill', trips_count=94379),
  Row(PUZone='Midtown East', trips_count=90605),
  Row(PUZone='Lincoln Square East', trips_count=85185),
  Row(PUZone='Clinton East', trips_count=81205),
  Row(PUZone='Upper West Side South', trips_count=79097)],
 [Row(PUZone='Newark Airport', average_unit_profit=279.3514957264957),
  Row(PUZone='Highbridge Park', average_unit_profit=74.38725490196077),
  Row(PUZone='Prospect Park', average_unit_profit=60.27407407407407),
  Row(PUZone='Sunset Park East', average_unit_profit=50.08473785387427),
  Row(PUZone='Columbia Street', average_unit_profit=42.464285714

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S        | M           | L           | XL          | XXL          |
|-----------------------------|----------|-------------|-------------|-------------|--------------|
| rows (M)                    |2,898,033 | 15,571,166  | 41,953,716  | 90,443,069  | 132,396,785  |
| execution time   (w/o 1.2)  | 16.39    | 2.21(min)   | 20.03(min)  | 26.07(min)  | 27.11(min)   |
| execution time              | 19.73    | 4.52(min)   | 32.59(min)  | 45.02(min)  | 44.16(min)   |
| sec / 1M records (w/o 1.2)  | 0.57     | 11.622      | 28.64       | 17.26       | 12.29        |
| sec / 1M records            | 0.68     | 17.41       | 46.70       | 29.85       | 20.00        |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S          | M          | L          | XL         | XXL          |
|-----------------------------|------------|------------|------------|------------|--------------|
| rows (M)                    | 2,898,033  | 15,571,166 | 41,953,716 | 90,443,069 | 132,396,785  |
| execution time   (w/o 1.2)  | 13.93      | 25.87      | 29.88      | 31.25      | 35.48(sec)   |
| execution time              | 26.11      | 25.11      | 45.49      | 24.10      | 1.15(min)    |
| sec / 1M records (w/o 1.2)  | 4.97       | 1.66       | 0.71       | 0.34       | 0.27         |
| sec / 1M records            | 9.325      | 1.61       | 1.09       | 0.27       | 0.57         |